<a href="https://colab.research.google.com/github/gcoelho3008/modelo_regressao_linear_simples/blob/main/modelo_pontua%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir dados

In [ ]:
! mv *.csv dados/

In [ ]:
! pip install pingouin

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import shapiro, kstest, probplot, zscore
from statsmodels.graphics.gofplots import qqplot
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import root_mean_squared_error
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pingouin as pg


Carga dos Dados

In [ ]:
df_pontuacao = pd.read_csv('./dados/pontuacao_test.csv')
df_pontuacao.head()

EDA

In [ ]:
df_pontuacao.describe()

In [ ]:
# Plot de dispersão x horas_estudo, y pontuação teste
sns.scatterplot(data=df_pontuacao, x='horas_estudo', y='pontuacao_teste')


In [ ]:
# verificar outliers
# grafico oxplot
sns.boxplot(df_pontuacao, y='horas_estudo')

In [ ]:
# verificar outliers
# grafico oxplot
sns.boxplot(df_pontuacao, y='pontuacao_teste')

In [ ]:
# vericar correlação = Pearson
sns.heatmap(df_pontuacao.corr('pearson'), annot=True)

In [ ]:
# vericar correlação = Spearman
sns.heatmap(df_pontuacao.corr('spearman'), annot=True)

In [ ]:
# Histograma das variáveis
sns.displot(df_pontuacao['horas_estudo'])

In [ ]:
# Histograma das variáveis
sns.displot(df_pontuacao, x='pontuacao_teste')

Treinar modelo


In [ ]:
# Dividir entre treino e teste
# Quado temos apenas uma feature, precisamos ajustar o shape
X = df_pontuacao['horas_estudo'].values.reshape(-1, 1)
y = df_pontuacao['pontuacao_teste'].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

In [ ]:
# Instanciar o modelo a ser treinado
reg_model = LinearRegression()

In [ ]:
# Treiar o modelo
reg_model.fit(X_train, y_train)

In [ ]:
# imprimir a equação da reta
# y = ax + b
print('A eequação da reta é y = {:4f}x + {:4f}'.format(reg_model.coef_[0][0], reg_model.intercept_[0], reg_model.intercept_[0]))

### Validar modelo = métricas

In [ ]:
# Predição dos valores com base no conjunto de testes
y_pred = reg_model.predict(X_test)

In [ ]:
# Calcular métrica Rsquared ou coeficiente de Determiação, R2 representa a proporção na variável dependente que é explicada pela variável independente
r2_score(y_test, y_pred)

In [ ]:
# Calcular métrica MAE (mean asolute error)
mean_absolute_error(y_test, y_pred)

In [ ]:
# calcular métrica MSE (Mean Square Error)
mean_squared_error(y_test, y_pred)

In [ ]:
rmse = root_mean_squared_error(y_test, y_pred)
print(rmse)

In [ ]:
# Analise gráfica
x_axis = range(len(y_test))
plt.figure(figsize=(10, 6))
sns.scatterplot(x=x_axis, y=y_test.reshape(-1), color='blue', label='Valores Reais')
sns.scatterplot(x=x_axis, y=y_pred.reshape(-1), color='red', label='Valores Preditos')
plt.legend()
plt.show()

Análise de Residuos

In [ ]:
# Calcular Residuos
residuos = y_test - y_pred


In [ ]:
# calcular os residuos padronizados (stsndssrdizatio)
# Para cada elemento de um conjunto (x = media) / deesvio padrão
residuos_std = zscore(residuos)

In [ ]:
"""
 análise gráfico de resíduos
 Verificar linearidadde do modelo:
 Se os resíduos estiverem entre -2 e 2 (na escala padrão) = indica linearidade
 Verificar homogeneidade das variancias (Homocedasticidade)
 Se tivermos alguma tendencia ou padrão (formam um cone, funil) há heterocefasticidade

 """
sns.scatterplot(x=y_pred.reshape(-1), y=residuos_std.reshape(-1))
plt.axhline(y=0, color='red')


In [ ]:
"""
Checar se resídups seguem uma distribuição normal
QQ (Quantile-Quantile) Plot, que avalia de uma maostra segue uma distribuiação normal
"""
pg.qqplot(residuos_std, dist='norm', confidence=0.95)
plt.xlabel('Quantis Teóricos')
plt.ylabel('Residuos na escala padrão')
plt.show()